# System Status

View the current status of:
- Active cycle
- Step progress
- Recent activity
- System health

In [5]:
from system import cycle, database as db, ux
from pathlib import Path

In [ ]:
ux.header("System Health")

health_checks = []

# Database connection
if db.test_connection():
    health_checks.append(['Database', ' Connected'])
else:
    health_checks.append(['Database', ' Not Connected'])

# Template directory
template_path = Path('home/jovyan/workspace/workflows/_Template')
if template_path.exists():
    health_checks.append(['Template', ' Found'])
else:
    health_checks.append(['Template', '  Missing'])

# Active cycle
if active_cycle:
    health_checks.append(['Active Cycle', f" {active_cycle['cycle_name']}"])
else:
    health_checks.append(['Active Cycle', '  None'])

ux.table(health_checks, headers=['Component', 'Status'])

print("\nQuick Actions:")
print("  • Create new cycle: Run 'workflow/_Tools/Create New Cycle.ipynb'")
print("  • Initialize database: Run 'workflow/_Tools/Database Admin.ipynb'")
print("  • View step details: Navigate to workflows/Active_<cycle-name> directory")

Component,Status
Database,Connected
Template,Missing
Active Cycle,Q4_2025_Demo-Analysis
System Lock,Initialized



Quick Actions:
  • Create new cycle: Run 'workflow/_Tools/Create New Cycle.ipynb'
  • Initialize database: Run 'workflow/_Tools/Database Admin.ipynb'
  • View step details: Navigate to workflows/Active_<cycle-name> directory





- [Initialize Database](./Database_Admin.ipynb)

---

In [3]:
ux.subheader("Active Cycle")

active_cycle = db.get_active_cycle()

if active_cycle:
    ux.cycle_summary(active_cycle)
    
    # Get progress stats
    cycle_name = active_cycle['cycle_name']
    
    # Count completed steps
    query = """
        SELECT COUNT(DISTINCT st.id) as completed_steps
        FROM irp_step st
        INNER JOIN irp_stage sg ON st.stage_id = sg.id
        INNER JOIN irp_cycle c ON sg.cycle_id = c.id
        WHERE c.cycle_name = %s
        AND EXISTS (
            SELECT 1 FROM irp_step_run sr 
            WHERE sr.step_id = st.id AND sr.status = 'completed'
        )
    """
    completed = db.execute_scalar(query, (cycle_name,))
    
    # Total steps
    query = """
        SELECT COUNT(*) FROM irp_step st
        INNER JOIN irp_stage sg ON st.stage_id = sg.id
        INNER JOIN irp_cycle c ON sg.cycle_id = c.id
        WHERE c.cycle_name = %s
    """
    total = db.execute_scalar(query, (cycle_name,))
    
    ux.subheader("Progress Overview")
    ux.pgbar(completed or 0, total or 0, title="Overall Completion")
    
else:
    ux.warning("No active cycle")
    print("\nRun 'Create New Cycle.ipynb' to create a new cycle")

Property,Value
Status,active
Created,2025-10-10 17:44:24.341276+00:00
Created By,notebook_user


In [ ]:
if active_cycle:
    ux.header(f"Step Progress: {active_cycle['cycle_name']}")
    
    progress_df = cycle.get_cycle_progress(active_cycle['cycle_name'])
    
    if not progress_df.empty:
        ux.step_progress(progress_df)
        
        # Summary statistics
        ux.subheader("Summary")
        
        status_counts = progress_df['last_status'].value_counts(dropna=False)
        
        total_steps = len(progress_df)
        completed = status_counts.get('completed', 0)
        running = status_counts.get('running', 0)
        failed = status_counts.get('failed', 0)
        skipped = status_counts.get('skipped', 0)
        not_started = len(progress_df[progress_df['last_status'].isna()])
        
        summary_data = [
            ['Total Steps', total_steps],
            [' Completed', completed],
            [' Running', running],
            [' Failed', failed],
            [' Skipped', skipped],
            [' Not Started', not_started]
        ]
        
        ux.table(summary_data, headers=['Status', 'Count'])
    else:
        ux.warning("No steps registered for this cycle")

In [ ]:
if active_cycle:
    ux.header("Recent Activity")
    
    history_df = cycle.get_step_history(active_cycle['cycle_name'])
    
    if not history_df.empty:
        # Format duration column
        history_df['duration'] = history_df['duration_seconds'].apply(ux.format_duration)
        
        # Select columns to display
        display_df = history_df[[
            'stage_name', 'step_name', 'run_number', 'status', 
            'started_ts', 'duration'
        ]].head(10)
        
        ux.dataframe(display_df, title="Last 10 Step Runs")
    else:
        ux.info("No step runs yet")

In [ ]:
if active_cycle:
    ux.header("Active Cycle Directory")
    
    active_dir = Path(f"home/jovyan/workspace/workflows/Active_{active_cycle['cycle_name']}")
    
    if active_dir.exists():
        print(f"/ {active_dir}\n")
        
        # Show directory tree
        for item in sorted(active_dir.iterdir()):
            if item.is_dir():
                print(f"  / {item.name}/")
                
                # Show subdirectories for notebooks
                if item.name == 'notebooks':
                    for stage_dir in sorted(item.iterdir()):
                        if stage_dir.is_dir() and not stage_dir.name.startswith('.'):
                            print(f"      / {stage_dir.name}/")
                            
                            # Show step notebooks
                            for step_file in sorted(stage_dir.iterdir()):
                                if step_file.suffix == '.ipynb' and not step_file.name.startswith('.'):
                                    print(f"          - {step_file.name}")
            
            elif item.suffix in ['.md', '.txt'] and not item.name.startswith('.'):
                print(f"  - {item.name}")
    else:
        ux.error(f"Directory not found: {active_dir}")




- [Initialize Database](./Database_Admin.ipynb)

---